In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [97]:
df=pd.read_pickle('co_police_arrests.pkl')

In [104]:
df.head()

,county_name,police_department,driver_gender,driver_race,violation,search_conducted,search_type,stop_outcome,is_arrested
0,Arapahoe County,Golden,F,White,Lights,False,NaN,Verbal Warning,False
1,El Paso County,Colorado Springs,F,White,Speeding,False,NaN,Verbal Warning,False
2,Garfield County,Glenwood Springs,M,White,Safe movement,False,NaN,Verbal Warning,False
3,Yuma County,Sterling,M,White,License,False,NaN,Citation,False
4,Jefferson County,6A,F,Hispanic,Lights,False,NaN,Written Warning,False


In [110]:
race = df.groupby('driver_race')
for r,group in race:
    

KeyError: <pandas.core.groupby.DataFrameGroupBy object at 0x10b55ca50>

In [107]:
race_arrest.groupby('driver_race')